In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# get links for each categories
def category_link():
    url = 'http://books.toscrape.com/'
    response = requests.get(
    "http://books.toscrape.com/index.html")
    soup = BeautifulSoup(response.text, "html.parser")
    tag_name = 'li a'
    articles = soup.select(tag_name)
    category = {}
    for art in articles:
        category[art.text.strip()] = url + art['href']
    return category

In [3]:
# get links for each books
def books_link(lookup,category):
    books={}
    for i in lookup:
        url = category.get(i)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        tag_name = 'h3 a'
        articles = soup.select(tag_name)

        for art in articles:
            books[art['title']] = ['http://books.toscrape.com/catalogue/' + art['href'][9:],i]

    return books


In [4]:
# get book description and column names
def find_des(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    tag_name = 'p'
    articles = soup.select(tag_name)
    col=[]
    for art in articles:
        a = art.text
        col.append(a)
    prod_des = max(col,key=len)

    tag_name = 'tr th'
    articles = soup.select(tag_name)
    col_name = []
    for art in articles:
        a=art.text
        col_name.append(a)
    col_name = ['Category'] + ['Book Name'] + col_name + ['Product Description']

    return prod_des, col_name

In [5]:
# get book info
def find_info(books_titles):
    des = []
    for i in books_titles:
        url = books.get(i)[0]
        cat = books.get(i)[1]
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        tag_name = 'tr td'
        articles = soup.select(tag_name)

        des_temp = []
        for art in articles:
            a= art.text
            des_temp.append(a)
        prod_des, col_name = find_des(url)
        des_temp = [cat] + [i] + des_temp + [prod_des]
        des.append(des_temp)
    return des, col_name



In [11]:
# Main
lookup = ['Science' , 'Poetry']
category = category_link()
books = books_link(lookup,category)
books_titles = list(books.keys())
des, col_name = find_info(books_titles)

# create a DataFrame and save as excel file
df = pd.DataFrame(des,columns=col_name)
df.to_excel('Report.xlsx')

